In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os,sys
from collections import defaultdict

from functions_transposons_outside_genes import get_genes_names
from functions_transposons_outside_genes import defining_threshold_given_tr_density
from functions_transposons_outside_genes import get_discarded_genes_by_duplication
from functions_transposons_outside_genes import get_amenable_genes_coverage_neighborhood
from functions_transposons_outside_genes import local_discrimination_genes_by_neighbors_coverage

In [3]:
data_all=pd.read_excel('../postprocessed-data/postprocessed_data_all_backgrounds.xlsx',engine="openpyxl")

In [4]:

keys=['dbem3_b',
 'dnrp1_b',
 'bem1-aid_a',
 'dnrp1_a',
 'dbem1dbem3_b',
 'wt_merged',
 'dbem1dbem3_a',
 'bem1-aid-dbem3_a',
 'bem1-aid-dbem3_b',
 'wt_b',
 'wt_a',
 'dnrp1_merged',
 'bem1-aid_b',
 'dbem3_merged',
 'dbem3_a']

In [5]:
from ast import literal_eval
positions_float_pd=pd.read_csv("../postprocessed-data/genetic_positions_float_pd_all_backgrounds.csv",converters={'Positions_float': literal_eval,'Ninsertions': literal_eval})
positions_float_pd.rename(columns={'Unnamed: 0':'Gene name', "Unnamed: 1": "background"},inplace=True)

In [6]:
data_all_modified,genes_names=get_genes_names(data_raw=data_all,keys=keys)

In [7]:

## getting genes that are duplicated in the genome 
discarded_genes_by_duplication=get_discarded_genes_by_duplication(positions_float_pd,genes_names=genes_names)



In [8]:
## Getting amenable genes for analysis of local neighborhood

# Independent variables 

windows_size=10000
targets,genes_not_discarded_by_location=get_amenable_genes_coverage_neighborhood(positions_float_pd,genes_names=genes_names,discarded_genes_by_duplication=discarded_genes_by_duplication,windows_size=windows_size)


In [17]:
genes_not_discarded_by_location

array(['COX26', 'CUB1', 'ERG29', 'KCH1', 'MCY1', 'MPO1', 'NVJ2', 'PXP1',
       'PXP2'], dtype=object)

In [25]:
background="wt_merged"
## defining the Threshold for each background
threshold,density=defining_threshold_given_tr_density(data_all_modified,windows_size=windows_size,background=background)
## Discriminating genes by local neighborhood coverage
n=0
k=10
a,b,c,d=local_discrimination_genes_by_neighbors_coverage(positions_float_pd,background=background,gene_of_interest="BEM1",windows_size=windows_size,threshold=k*threshold)


In [21]:
a,b,c,d,k*threshold,density

([], [], 21322, 194613, 4140.0, 0.04142334392876978)

In [26]:
## Big loop to get the local neighborhood coverage for each background 
background="wt_merged"
k=10 # Amplified factor for the threshold
for amenable_genes in targets:
    genes_out=[]
    a,b,c,d=local_discrimination_genes_by_neighbors_coverage(positions_float_pd,background=background,gene_of_interest=amenable_genes,windows_size=windows_size,threshold=k*threshold)
    if a!=[]:
        genes_out.append(a)
    if b!=[]:
        genes_out.append(b)